In [16]:
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import json

SetLogLevel(0)

Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7fb40e77fb90> (for post_run_cell):


ImportError: cannot import name 'configure_inline_support_orig' from 'matplotlib_inline.backend_inline' (/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/matplotlib_inline/backend_inline.py)

# Read audiofile


In [2]:
model_vosk = Model("model")

In [3]:
%%time

wf = wave.open("untitled.wav", "rb")
if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
    print ("Audio file must be WAV format mono PCM.")
    exit (1)

rec = KaldiRecognizer(model_vosk, wf.getframerate())

results = []

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        #print(rec.Result())
        jj = json.loads(rec.Result())["text"]
        print(jj)
        results.append( jj )
        pass
    else:
        #print(rec.PartialResult())
        pass

#print(rec.FinalResult())
j = json.loads( rec.FinalResult() )
print(j['text'])
for r in results:
    print(r)

tiens et à opposer la figure de mme de maintenon à celle de ninon de lenclos d'un côté la femme de lettres galantes et libertines
et de l autre la femme de pouvoir secrète et des votes
le mythe du grand siècle dont nous sommes les héritiers articule ainsi une image de l'écriture féminine
tiens et à opposer la figure de mme de maintenon à celle de ninon de lenclos d'un côté la femme de lettres galantes et libertines
et de l autre la femme de pouvoir secrète et des votes
CPU times: user 3.39 s, sys: 21.6 ms, total: 3.41 s
Wall time: 3.41 s


# load GPT-2 model 

* load gpt model from our checkpoint
* load tokenizer from our checkpoint
* put it on the gpu
* set it to eval mode

In [4]:
# https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=hQ1oK0kXaV5p
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

# Load pretrained model and tokenizer
model_test = GPT2LMHeadModel.from_pretrained("results-finetune/checkpoint-2500/")
tokenizer = GPT2TokenizerFast.from_pretrained("antoiloui/belgpt2", model_max_length=768, pad_token='<|pad|>')

device = torch.device("cuda")

model_test.cuda()

# Generate a sample of text
model_test.eval()


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

# make prompt from vosk results

In [9]:
%%time

for_wave_rnn = []

for q in results:
    prompt = "{}A: {}\n B: ".format(tokenizer.bos_token, q)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)

    output = model_test.generate(
                inputs,
                do_sample=True,   
                top_k=50, 
                max_length=100,
                top_p=0.95, 
                num_return_sequences=2,
                pad_token_id=tokenizer.eos_token_id
    )

    # Decode it
    decoded_output = []
    for sample in output:
        decoded_output.append(tokenizer.decode(sample, skip_special_tokens=True))
    #print(decoded_output)
    for o in decoded_output:
        b = o.split( 'B:' )[1]
        print(b)
        for_wave_rnn.append(b + "|00-de|fr")



  Ouais… (Un peu embarrassée.) Ah! Si tu as… beaucoup de respect pour ce que tu viens de me répondre, sois poli, c’est juste un peu plus que ça…
  Non, non. (Elles rient de bon cœur.)
  (Agacé.) Ben… C’est vrai que je ne suis pas dupe. Donc, si on peut lui faire confiance, n’est-ce pas lui qui nous a trompées?
  A, qui a toutes les faveurs du gouverneur en scène, y a-t-il d’autres questions au fin fond de votre cœur?
CPU times: user 3.96 s, sys: 0 ns, total: 3.96 s
Wall time: 3.96 s
Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7fb40e77fb90> (for post_run_cell):


ImportError: cannot import name 'configure_inline_support_orig' from 'matplotlib_inline.backend_inline' (/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/matplotlib_inline/backend_inline.py)

# load wavernn and taco model

In [10]:
os.chdir(os.path.expanduser("~"))
    
tacotron_dir = "Multilingual_Text_to_Speech"
wavernn_dir = "WaveRNN"

os.chdir(os.path.join(os.path.expanduser("~"), "checkpoints"))

tacotron_chpt = "generated_switching.pyt"
wavernn_chpt = "wavernn_weight.pyt"

os.chdir(os.path.expanduser("~"))

os.chdir(os.path.join(os.path.expanduser("~"), tacotron_dir))
if "utils" in sys.modules: del sys.modules["utils"]

from synthesize import synthesize
from utils import build_model

model_taco = build_model(os.path.join(os.path.expanduser("~"), "checkpoints", tacotron_chpt))
model_taco.eval()

os.chdir(os.path.join(os.path.expanduser("~"), wavernn_dir))
if "utils" in sys.modules: del sys.modules["utils"]

from models.fatchord_version import WaveRNN
from utils import hparams as hp
from gen_wavernn import generate
import torch

hp.configure('hparams.py')
model_wave = WaveRNN(rnn_dims=hp.voc_rnn_dims, fc_dims=hp.voc_fc_dims, bits=hp.bits, pad=hp.voc_pad, upsample_factors=hp.voc_upsample_factors, 
                feat_dims=hp.num_mels, compute_dims=hp.voc_compute_dims, res_out_dims=hp.voc_res_out_dims, res_blocks=hp.voc_res_blocks, 
                hop_length=hp.hop_length, sample_rate=hp.sample_rate, mode=hp.voc_mode).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model_wave.load(os.path.join(os.path.expanduser("~"), "checkpoints", wavernn_chpt))




Trainable Parameters: 4.744M
Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7fb40e77fb90> (for post_run_cell):


ImportError: cannot import name 'configure_inline_support_orig' from 'matplotlib_inline.backend_inline' (/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/matplotlib_inline/backend_inline.py)

# do the heavy work and generate sound

In [15]:
%%time
import IPython
spectrograms = [synthesize(model_taco, "|" + i) for i in for_wave_rnn]
waveforms = [generate(model_wave, s, hp.voc_gen_batched, hp.voc_target, hp.voc_overlap) for s in spectrograms]

for idx, w in enumerate(waveforms):
  print(for_wave_rnn[idx])
  IPython.display.display(IPython.display.Audio(data=w, rate=hp.sample_rate))

| ████████████████ 144000/145200 | Batch Size: 12 | Gen Rate: 11.0kHz |   Ouais… (Un peu embarrassée.) Ah! Si tu as… beaucoup de respect pour ce que tu viens de me répondre, sois poli, c’est juste un peu plus que ça…|00-de|fr


  Non, non. (Elles rient de bon cœur.)|00-de|fr


  (Agacé.) Ben… C’est vrai que je ne suis pas dupe. Donc, si on peut lui faire confiance, n’est-ce pas lui qui nous a trompées?|00-de|fr


  A, qui a toutes les faveurs du gouverneur en scène, y a-t-il d’autres questions au fin fond de votre cœur?|00-de|fr


CPU times: user 1min, sys: 132 ms, total: 1min
Wall time: 1min
Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x7fb40e77fb90> (for post_run_cell):


ImportError: cannot import name 'configure_inline_support_orig' from 'matplotlib_inline.backend_inline' (/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/matplotlib_inline/backend_inline.py)